# Spatial Weights

Spatial weights are mathematical structures used to represent spatial relationships. They characterize the relationship of each observation to every other observation using some concept of proximity or closeness that depends on the weight type.

They can be build in PySAL from shapefiles, as well as some types of files. 

In [1]:
import pysal as ps
import numpy as np

There are functions to construct weights directly from a file path. 

In [2]:
shp_path = ps.examples.get_path('NAT.shp')

## Weight Types

### Contiguity: 
#### Queen Weights

A commonly-used type of weight is a queen contigutiy weight, which reflects adjacency relationships as a binary indicator variable denoting whether or not a polygon shares an edge or a verted each another polygon. These weights are symmetric, in that when polygon $A$ neighbors polygon $B$, both $w_{AB} = 1$ and $w_{BA} = 1$.

To construct queen weights from a shapefile, use the `queen_from_shapefile` function:

In [3]:
qW = ps.queen_from_shapefile(shp_path)
dataframe = ps.pdio.read_files(shp_path)

In [4]:
qW

All weights objects have a few traits that you can use to work with the weights object, as well as to get information about the weights object. 

To get the neighbors & weights around an observation, use the observation's index on the weights object, like a dictionary:

In [5]:
qW[4] #neighbors & weights of the 5th observation

{2: 1.0, 5: 1.0, 28: 1.0, 62: 1.0}

By default, the weights and the pandas dataframe will use the same index. So, we can view the observation and its neighbors in the dataframe by putting the observation's index and its neighbors' indexes together in one list:

In [6]:
self_and_neighbors = [4]
self_and_neighbors.extend(qW.neighbors[4])
print(self_and_neighbors)

[4, 2, 28, 5, 62]


and grabbing those elements from the dataframe:

In [7]:
dataframe.loc[self_and_neighbors]

,NAME,STATE_NAME,STATE_FIPS,CNTY_FIPS,FIPS,STFIPS,COFIPS,FIPSNO,SOUTH,HR60,...,BLK90,GI59,GI69,GI79,GI89,FH60,FH70,FH80,FH90,geometry
4,Pend Oreille,Washington,53,051,53051,53,51,53051,0,0.000000,...,0.134605,0.243263,0.365614,0.358706,0.387848,8.243930,4.1,7.557643,10.313002,<pysal.cg.shapes.Polygon object at 0x7f1756683...
2,Stevens,Washington,53,065,53065,53,65,53065,0,1.863863,...,0.210030,0.283999,0.394083,0.357566,0.369942,9.258437,5.6,6.812127,10.352015,<pysal.cg.shapes.Polygon object at 0x7f1756683...
28,Bonner,Idaho,16,017,16017,16,17,16017,0,2.138534,...,0.138983,0.261589,0.367973,0.370304,0.375177,8.661813,7.1,8.324631,9.571821,<pysal.cg.shapes.Polygon object at 0x7f17566c7...
5,Boundary,Idaho,16,021,16021,16,21,16021,0,0.000000,...,0.036006,0.261939,0.350351,0.355913,0.340525,7.112971,6.8,8.249497,9.343201,<pysal.cg.shapes.Polygon object at 0x7f1756683...
62,Spokane,Washington,53,063,53063,53,63,53063,0,2.514973,...,1.412703,0.251340,0.346762,0.356968,0.370488,10.099520,10.2,12.687751,15.229159,<pysal.cg.shapes.Polygon object at 0x7f1756936...


A full, dense matrix describing all of the pairwise relationships is constructed using the `.full` method, or when `pysal.full` is called on a weights object:

In [8]:
Wmatrix, ids = qW.full()
#Wmatrix, ids = ps.full(qW)

In [9]:
Wmatrix

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

Note that this matrix is binary, in that its elements are either zero or one, since an observation is either a neighbor or it is not a neighbor. 

However, many common use cases of spatial weights require that the matrix is row-standardized. This is done simply in PySAL using the `.transform` attribute

In [10]:
qW.transform = 'r'

Now, if we build a new full matrix, its rows should sum to one:

In [11]:
Wmatrix, ids = qW.full()

In [12]:
Wmatrix.sum(axis=1) #numpy axes are 0:column, 1:row, 2:facet, into higher dimensions

array([ 1.,  1.,  1., ...,  1.,  1.,  1.])

Since weight matrices are typically very sparse, there is also a sparse weights matrix constructor:

In [13]:
qW.sparse

<3085x3085 sparse matrix of type '<class 'numpy.float64'>'
	with 18168 stored elements in Compressed Sparse Row format>

By default, PySAL assigns each observation an index according to the order in which the observation was read in. This means that, by default, all of the observations in the weights object are indexed by table order. If you have an alternative ID variable, you can pass that into the weights constructor. 

For example, the `NAT.shp` dataset has a possible alternative ID Variable, a `FIPS` code.

In [14]:
dataframe.head()

,NAME,STATE_NAME,STATE_FIPS,CNTY_FIPS,FIPS,STFIPS,COFIPS,FIPSNO,SOUTH,HR60,...,BLK90,GI59,GI69,GI79,GI89,FH60,FH70,FH80,FH90,geometry
0,Lake of the Woods,Minnesota,27,077,27077,27,77,27077,0,0.000000,...,0.024534,0.285235,0.372336,0.342104,0.336455,11.279621,5.4,5.663881,9.515860,<pysal.cg.shapes.Polygon object at 0x7f1756683...
1,Ferry,Washington,53,019,53019,53,19,53019,0,0.000000,...,0.317712,0.256158,0.360665,0.361928,0.360640,10.053476,2.6,10.079576,11.397059,<pysal.cg.shapes.Polygon object at 0x7f1756683...
2,Stevens,Washington,53,065,53065,53,65,53065,0,1.863863,...,0.210030,0.283999,0.394083,0.357566,0.369942,9.258437,5.6,6.812127,10.352015,<pysal.cg.shapes.Polygon object at 0x7f1756683...
3,Okanogan,Washington,53,047,53047,53,47,53047,0,2.612330,...,0.155922,0.258540,0.371218,0.381240,0.394519,9.039900,8.1,10.084926,12.840340,<pysal.cg.shapes.Polygon object at 0x7f1756683...
4,Pend Oreille,Washington,53,051,53051,53,51,53051,0,0.000000,...,0.134605,0.243263,0.365614,0.358706,0.387848,8.243930,4.1,7.557643,10.313002,<pysal.cg.shapes.Polygon object at 0x7f1756683...


The observation we were discussing above is in the fifth row: Pend Oreille county, Washington. Note that its FIPS code is 53051.

Then, instead of indexing the weights and the dataframe just based on read-order, use the `FIPS` code as an index:

In [15]:
qW = ps.queen_from_shapefile(shp_path, idVariable='FIPS')

In [16]:
ps.knnW_from_shapefile??

Now, Pend Oreille county has a different index:

In [17]:
qW[4] #fails, since no FIPS is 4. 

KeyError: 4

Note that a `KeyError` in Python usually means that some index, here `4`, was not found in the collection being searched, the IDs in the queen weights object. This makes sense, since we explicitly passed an `idVariable` argument, and nothing has a `FIPS` code of 4.

Instead, if we use the observation's `FIPS` code:

In [18]:
qW['53051']

{'16017': 1.0, '16021': 1.0, '53063': 1.0, '53065': 1.0}

We get what we need.

In addition, we have to now query the dataframe using the `FIPS` code to find our neighbors. But, this is relatively easy to do, since pandas will parse the query by looking into python objects, if told to. 

First, let us store the neighbors of our target county:

In [19]:
self_and_neighbors = ['53051']
self_and_neighbors.extend(qW.neighbors['53051'])

Then, we can use this list in `.query`: 

In [20]:
dataframe.query('FIPS in @self_and_neighbors')

,NAME,STATE_NAME,STATE_FIPS,CNTY_FIPS,FIPS,STFIPS,COFIPS,FIPSNO,SOUTH,HR60,...,BLK90,GI59,GI69,GI79,GI89,FH60,FH70,FH80,FH90,geometry
2,Stevens,Washington,53,065,53065,53,65,53065,0,1.863863,...,0.210030,0.283999,0.394083,0.357566,0.369942,9.258437,5.6,6.812127,10.352015,<pysal.cg.shapes.Polygon object at 0x7f1756683...
4,Pend Oreille,Washington,53,051,53051,53,51,53051,0,0.000000,...,0.134605,0.243263,0.365614,0.358706,0.387848,8.243930,4.1,7.557643,10.313002,<pysal.cg.shapes.Polygon object at 0x7f1756683...
5,Boundary,Idaho,16,021,16021,16,21,16021,0,0.000000,...,0.036006,0.261939,0.350351,0.355913,0.340525,7.112971,6.8,8.249497,9.343201,<pysal.cg.shapes.Polygon object at 0x7f1756683...
28,Bonner,Idaho,16,017,16017,16,17,16017,0,2.138534,...,0.138983,0.261589,0.367973,0.370304,0.375177,8.661813,7.1,8.324631,9.571821,<pysal.cg.shapes.Polygon object at 0x7f17566c7...
62,Spokane,Washington,53,063,53063,53,63,53063,0,2.514973,...,1.412703,0.251340,0.346762,0.356968,0.370488,10.099520,10.2,12.687751,15.229159,<pysal.cg.shapes.Polygon object at 0x7f1756936...


Note that we have to use `@` before the name in order to show that we're referring to a python object and not a column in the dataframe. 

In [21]:
#dataframe.query('FIPS in neighs') will fail because there is no column called 'neighs'

Of course, we could also reindex the dataframe to use the same index as our weights:

In [22]:
fips_frame = dataframe.set_index(dataframe.FIPS)
fips_frame.head()

,NAME,STATE_NAME,STATE_FIPS,CNTY_FIPS,FIPS,STFIPS,COFIPS,FIPSNO,SOUTH,HR60,...,BLK90,GI59,GI69,GI79,GI89,FH60,FH70,FH80,FH90,geometry
FIPS,,,,,,,,,,,,,,,,,,,,,
27077,Lake of the Woods,Minnesota,27,077,27077,27,77,27077,0,0.000000,...,0.024534,0.285235,0.372336,0.342104,0.336455,11.279621,5.4,5.663881,9.515860,<pysal.cg.shapes.Polygon object at 0x7f1756683...
53019,Ferry,Washington,53,019,53019,53,19,53019,0,0.000000,...,0.317712,0.256158,0.360665,0.361928,0.360640,10.053476,2.6,10.079576,11.397059,<pysal.cg.shapes.Polygon object at 0x7f1756683...
53065,Stevens,Washington,53,065,53065,53,65,53065,0,1.863863,...,0.210030,0.283999,0.394083,0.357566,0.369942,9.258437,5.6,6.812127,10.352015,<pysal.cg.shapes.Polygon object at 0x7f1756683...
53047,Okanogan,Washington,53,047,53047,53,47,53047,0,2.612330,...,0.155922,0.258540,0.371218,0.381240,0.394519,9.039900,8.1,10.084926,12.840340,<pysal.cg.shapes.Polygon object at 0x7f1756683...
53051,Pend Oreille,Washington,53,051,53051,53,51,53051,0,0.000000,...,0.134605,0.243263,0.365614,0.358706,0.387848,8.243930,4.1,7.557643,10.313002,<pysal.cg.shapes.Polygon object at 0x7f1756683...


Now that both are using the same weights, we can use the `.loc` indexer again:

In [23]:
fips_frame.loc[self_and_neighbors]

,NAME,STATE_NAME,STATE_FIPS,CNTY_FIPS,FIPS,STFIPS,COFIPS,FIPSNO,SOUTH,HR60,...,BLK90,GI59,GI69,GI79,GI89,FH60,FH70,FH80,FH90,geometry
FIPS,,,,,,,,,,,,,,,,,,,,,
53051,Pend Oreille,Washington,53,051,53051,53,51,53051,0,0.000000,...,0.134605,0.243263,0.365614,0.358706,0.387848,8.243930,4.1,7.557643,10.313002,<pysal.cg.shapes.Polygon object at 0x7f1756683...
53065,Stevens,Washington,53,065,53065,53,65,53065,0,1.863863,...,0.210030,0.283999,0.394083,0.357566,0.369942,9.258437,5.6,6.812127,10.352015,<pysal.cg.shapes.Polygon object at 0x7f1756683...
16017,Bonner,Idaho,16,017,16017,16,17,16017,0,2.138534,...,0.138983,0.261589,0.367973,0.370304,0.375177,8.661813,7.1,8.324631,9.571821,<pysal.cg.shapes.Polygon object at 0x7f17566c7...
16021,Boundary,Idaho,16,021,16021,16,21,16021,0,0.000000,...,0.036006,0.261939,0.350351,0.355913,0.340525,7.112971,6.8,8.249497,9.343201,<pysal.cg.shapes.Polygon object at 0x7f1756683...
53063,Spokane,Washington,53,063,53063,53,63,53063,0,2.514973,...,1.412703,0.251340,0.346762,0.356968,0.370488,10.099520,10.2,12.687751,15.229159,<pysal.cg.shapes.Polygon object at 0x7f1756936...


#### Rook Weights

Rook weights are another type of contiguity weight, but consider observations as neighboring only when they share an edge. The rook neighbors of an observation may be different than its queen neighbors, depending on how the observation and its nearby polygons are configured. 

We can construct this in the same way as the queen weights, using the special `rook_from_shapefile` function

In [24]:
rW = ps.rook_from_shapefile(shp_path, idVariable='FIPS')

In [25]:
rW['53051']

{'16017': 1.0, '16021': 1.0, '53063': 1.0, '53065': 1.0}

These weights function exactly like the Queen weights, and are only distinguished by what they consider "neighbors."

#### Bishop Weights

In theory, a "Bishop" weighting scheme is one that arises when only polygons that share vertexes are considered to be neighboring. But, since Queen contiguigy requires either an edge or a vertex and Rook contiguity requires only shared edges, the following relationship is true:

$$ \mathcal{Q} = \mathcal{R} \cup \mathcal{B} $$

where $\mathcal{Q}$ is the set of neighbor pairs *via* queen contiguity, $\mathcal{R}$ is the set of neighbor pairs *via* Rook contiguity, and $\mathcal{B}$ *via* Bishop contiguity. Thus:

$$ \mathcal{Q} \setminus \mathcal{R} = \mathcal{B}$$

Bishop weights entail all Queen neighbor pairs that are not also Rook neighbors.

PySAL does not have a dedicated bishop weights constructor, but you can construct very easily using the `w_difference` function. This function is one of a family of tools to work with weights, all defined in `ps.weights`, that conduct these types of set operations between weight objects.

In [26]:
bW = ps.w_difference(qW, rW, constrained=False, silent_island_warning=True) #silence because there will be a lot of warnings

In [27]:
bW.histogram

[(0, 2359), (1, 531), (2, 150), (3, 31), (4, 14)]

Thus, the vast majority of counties have no bishop neighbors. But, a few do. A simple way to see these observations in the dataframe is to find all elements of the dataframe that are not "islands," the term for an observation with no neighbors:

In [28]:
islands = bW.islands

In [29]:
dataframe.query('FIPS not in @islands')

,NAME,STATE_NAME,STATE_FIPS,CNTY_FIPS,FIPS,STFIPS,COFIPS,FIPSNO,SOUTH,HR60,...,BLK90,GI59,GI69,GI79,GI89,FH60,FH70,FH80,FH90,geometry
32,Williams,North Dakota,38,105,38105,38,105,38105,0,0.000000,...,0.085191,0.253422,0.331675,0.336543,0.362336,9.133037,8.300000,8.312172,9.884332,<pysal.cg.shapes.Polygon object at 0x7f17566c7...
35,Roosevelt,Montana,30,085,30085,30,85,30085,0,5.682948,...,0.154560,0.304054,0.376976,0.381225,0.391364,12.080537,11.500000,15.203586,21.936186,<pysal.cg.shapes.Polygon object at 0x7f17566c7...
56,McKenzie,North Dakota,38,053,38053,38,53,38053,0,13.706140,...,0.047000,0.302126,0.369761,0.363755,0.370844,11.775148,5.100000,5.312158,9.595655,<pysal.cg.shapes.Polygon object at 0x7f1756936...
58,Richland,Montana,30,083,30083,30,83,30083,0,0.000000,...,0.037327,0.289600,0.350725,0.336799,0.364463,9.497645,6.600000,5.822862,10.223426,<pysal.cg.shapes.Polygon object at 0x7f1756936...
123,Bayfield,Wisconsin,55,007,55007,55,7,55007,0,2.798769,...,0.207025,0.301325,0.369321,0.345692,0.360286,13.624070,5.500000,7.959937,10.115237,<pysal.cg.shapes.Polygon object at 0x7f1756d7c...
135,Douglas,Wisconsin,55,031,55031,55,31,55031,0,1.481218,...,0.407108,0.231009,0.327774,0.335744,0.354634,12.177288,11.100000,13.410263,16.076343,<pysal.cg.shapes.Polygon object at 0x7f1756d7c...
148,Emmons,North Dakota,38,029,38029,38,29,38029,0,0.000000,...,0.000000,0.338304,0.457558,0.436659,0.415981,8.798283,5.200000,5.485790,5.089606,<pysal.cg.shapes.Polygon object at 0x7f1756d7c...
153,Richland,North Dakota,38,077,38077,38,77,38077,0,0.000000,...,0.115715,0.319899,0.400422,0.344135,0.351533,11.422890,6.800000,6.506633,7.806045,<pysal.cg.shapes.Polygon object at 0x7f1756718...
166,Sioux,North Dakota,38,085,38085,38,85,38085,0,27.307482,...,0.079766,0.360412,0.411157,0.412360,0.430380,18.005952,16.800000,23.726542,30.778443,<pysal.cg.shapes.Polygon object at 0x7f1756718...
167,Pine,Minnesota,27,115,27115,27,115,27115,0,1.960323,...,1.655380,0.311982,0.365794,0.349111,0.364783,11.578691,6.000000,7.830749,9.040259,<pysal.cg.shapes.Polygon object at 0x7f1756718...


## Distance

There are many other kinds of weighting functions in PySAL. Another separate type use a continuous measure of distance to define neighborhoods. To use these measures, we first must extract the polygons' centroids.


For each polygon `poly` in `dataframe.geometry`, we want `poly.centroid`. So, one way to do this is to make a list of all of the centroids:

In [30]:
centroids = [list(poly.centroid) for poly in dataframe.geometry]

In [31]:
centroids[0:5] #let's look at the first five

[[-94.90336786329912, 48.771730563701574],
 [-118.51718120712802, 48.46959353253665],
 [-117.85532452342407, 48.39591039096631],
 [-119.73943524482668, 48.5484338901435],
 [-117.27400489644516, 48.53279719845048]]

 If we were working with point data, this step would be unncessary. 

### KnnW

If we wanted to consider only the `k`-nearest neighbors to an observation's centroid, we could use the `knnW` function in PySAL.

This specific type of distance weights requires that we first build a `KDTree`, a special representation for spatial point data. Fortunately, this is built in to PySAL:

In [32]:
kdtree = ps.cg.KDTree(centroids)

Then, we can use this to build a spatial weights object where only the closest `k` observations are considered "neighbors." In this example, let's do the closest 5:

In [33]:
nn5 = ps.knnW(kdtree, k=5)

In [34]:
nn5.histogram

[(5, 3085)]

So, all observations have exactly 5 neighbors. Sometimes, these neighbors are actually different observations than the ones identified by contiguity neighbors. 

For example, Pend Oreille gets a new neighbor, Kootenai county:

In [35]:
nn5[4]

{2: 1.0, 5: 1.0, 28: 1.0, 62: 1.0, 65: 1.0}

In [36]:
dataframe.loc[nn5.neighbors[4] + [4]]

,NAME,STATE_NAME,STATE_FIPS,CNTY_FIPS,FIPS,STFIPS,COFIPS,FIPSNO,SOUTH,HR60,...,BLK90,GI59,GI69,GI79,GI89,FH60,FH70,FH80,FH90,geometry
2,Stevens,Washington,53,065,53065,53,65,53065,0,1.863863,...,0.210030,0.283999,0.394083,0.357566,0.369942,9.258437,5.6,6.812127,10.352015,<pysal.cg.shapes.Polygon object at 0x7f1756683...
28,Bonner,Idaho,16,017,16017,16,17,16017,0,2.138534,...,0.138983,0.261589,0.367973,0.370304,0.375177,8.661813,7.1,8.324631,9.571821,<pysal.cg.shapes.Polygon object at 0x7f17566c7...
5,Boundary,Idaho,16,021,16021,16,21,16021,0,0.000000,...,0.036006,0.261939,0.350351,0.355913,0.340525,7.112971,6.8,8.249497,9.343201,<pysal.cg.shapes.Polygon object at 0x7f1756683...
62,Spokane,Washington,53,063,53063,53,63,53063,0,2.514973,...,1.412703,0.251340,0.346762,0.356968,0.370488,10.099520,10.2,12.687751,15.229159,<pysal.cg.shapes.Polygon object at 0x7f1756936...
65,Kootenai,Idaho,16,055,16055,16,55,16055,0,0.000000,...,0.134680,0.234269,0.343077,0.357464,0.365750,9.152454,8.8,9.351636,10.602422,<pysal.cg.shapes.Polygon object at 0x7f1756936...
4,Pend Oreille,Washington,53,051,53051,53,51,53051,0,0.000000,...,0.134605,0.243263,0.365614,0.358706,0.387848,8.243930,4.1,7.557643,10.313002,<pysal.cg.shapes.Polygon object at 0x7f1756683...


In [37]:
fips_frame.loc[qW.neighbors['53051'] + ['53051']]

,NAME,STATE_NAME,STATE_FIPS,CNTY_FIPS,FIPS,STFIPS,COFIPS,FIPSNO,SOUTH,HR60,...,BLK90,GI59,GI69,GI79,GI89,FH60,FH70,FH80,FH90,geometry
FIPS,,,,,,,,,,,,,,,,,,,,,
53065,Stevens,Washington,53,065,53065,53,65,53065,0,1.863863,...,0.210030,0.283999,0.394083,0.357566,0.369942,9.258437,5.6,6.812127,10.352015,<pysal.cg.shapes.Polygon object at 0x7f1756683...
16017,Bonner,Idaho,16,017,16017,16,17,16017,0,2.138534,...,0.138983,0.261589,0.367973,0.370304,0.375177,8.661813,7.1,8.324631,9.571821,<pysal.cg.shapes.Polygon object at 0x7f17566c7...
16021,Boundary,Idaho,16,021,16021,16,21,16021,0,0.000000,...,0.036006,0.261939,0.350351,0.355913,0.340525,7.112971,6.8,8.249497,9.343201,<pysal.cg.shapes.Polygon object at 0x7f1756683...
53063,Spokane,Washington,53,063,53063,53,63,53063,0,2.514973,...,1.412703,0.251340,0.346762,0.356968,0.370488,10.099520,10.2,12.687751,15.229159,<pysal.cg.shapes.Polygon object at 0x7f1756936...
53051,Pend Oreille,Washington,53,051,53051,53,51,53051,0,0.000000,...,0.134605,0.243263,0.365614,0.358706,0.387848,8.243930,4.1,7.557643,10.313002,<pysal.cg.shapes.Polygon object at 0x7f1756683...


#### Kernel W

Kernel Weights are continuous distance-based weights that use kernel densities to provide an indication of neighborliness. 

Typically, they estimate a `bandwidth`, which is a parameter governing how far out observations should be considered neighboring. Then, using this bandwidth, they evaluate a continuous kernel function to provide a weight between 0 and 1.

Many different choices of kernel functions are supported, and bandwidth can be estimated at each point or over the entire map.

For example, if we wanted to use a single estimated bandwidth for the entire map and weight according to a gaussian kernel:

In [38]:
kernelW = ps.Kernel(centroids, fixed=True, function='gaussian')
#ps.Kernel(centroids, fixed=False, function='gaussian') #same kernel, but bandwidth changes at each observation

In [39]:
dataframe.loc[kernelW.neighbors[4] + [4]]

,NAME,STATE_NAME,STATE_FIPS,CNTY_FIPS,FIPS,STFIPS,COFIPS,FIPSNO,SOUTH,HR60,...,BLK90,GI59,GI69,GI79,GI89,FH60,FH70,FH80,FH90,geometry
1,Ferry,Washington,53,019,53019,53,19,53019,0,0.000000,...,0.317712,0.256158,0.360665,0.361928,0.360640,10.053476,2.6,10.079576,11.397059,<pysal.cg.shapes.Polygon object at 0x7f1756683...
69,Lincoln,Washington,53,043,53043,53,43,53043,0,3.052783,...,0.169224,0.269758,0.336764,0.370721,0.368230,6.803193,4.7,4.343144,6.250000,<pysal.cg.shapes.Polygon object at 0x7f1756936...
2,Stevens,Washington,53,065,53065,53,65,53065,0,1.863863,...,0.210030,0.283999,0.394083,0.357566,0.369942,9.258437,5.6,6.812127,10.352015,<pysal.cg.shapes.Polygon object at 0x7f1756683...
110,Whitman,Washington,53,075,53075,53,75,53075,0,1.066223,...,1.263701,0.261340,0.358924,0.355758,0.381245,6.929178,4.8,7.927677,9.189189,<pysal.cg.shapes.Polygon object at 0x7f1756779...
62,Spokane,Washington,53,063,53063,53,63,53063,0,2.514973,...,1.412703,0.251340,0.346762,0.356968,0.370488,10.099520,10.2,12.687751,15.229159,<pysal.cg.shapes.Polygon object at 0x7f1756936...
4,Pend Oreille,Washington,53,051,53051,53,51,53051,0,0.000000,...,0.134605,0.243263,0.365614,0.358706,0.387848,8.243930,4.1,7.557643,10.313002,<pysal.cg.shapes.Polygon object at 0x7f1756683...
65,Kootenai,Idaho,16,055,16055,16,55,16055,0,0.000000,...,0.134680,0.234269,0.343077,0.357464,0.365750,9.152454,8.8,9.351636,10.602422,<pysal.cg.shapes.Polygon object at 0x7f1756936...
100,Benewah,Idaho,16,009,16009,16,9,16009,0,0.000000,...,0.075595,0.247637,0.339068,0.337666,0.354479,8.701299,3.9,7.240618,8.512545,<pysal.cg.shapes.Polygon object at 0x7f1756779...
28,Bonner,Idaho,16,017,16017,16,17,16017,0,2.138534,...,0.138983,0.261589,0.367973,0.370304,0.375177,8.661813,7.1,8.324631,9.571821,<pysal.cg.shapes.Polygon object at 0x7f17566c7...
5,Boundary,Idaho,16,021,16021,16,21,16021,0,0.000000,...,0.036006,0.261939,0.350351,0.355913,0.340525,7.112971,6.8,8.249497,9.343201,<pysal.cg.shapes.Polygon object at 0x7f1756683...


As you can see, this provides our target observation, Pend Oreille, with many new neighbors. \